<a href="https://colab.research.google.com/github/isaacyeos/nlp/blob/master/GPT_2_Text_Generating_Model_SOTU_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU For Free 

**This is not using TensorFlow 2.0 beta**   
You wil need TensorFlow >= 1.12 


In [1]:
!pip install -q tensorflow-gpu==1.13.1
!pip install -q gpt_2_simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

     |████████████████████████████████| 345.2MB 51kB/s 
     |████████████████████████████████| 3.2MB 33.3MB/s 
     |████████████████████████████████| 368kB 40.4MB/s 
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

## Verify GPU
 

In [2]:
!nvidia-smi

Mon Jun  1 16:02:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are two sizes of GPT-2:

* `117M` (default): the "small" model, 500MB on disk.
* `345M`: the "medium" model, 1.5GB on disk.

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [4]:
gpt2.download_gpt2(model_name="345M")

Fetching checkpoint: 1.05Mit [00:00, 441Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 88.8Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 540Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:14, 97.7Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 436Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 90.5Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 146Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [5]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Upload your textfile to colab

In [15]:
import os
sotu_path = '/content/drive/My Drive/0-software_dev_notes/dsta-ai-course/state-of-the-union-corpus-1989-2017/sotu/'
entries = os.listdir(sotu_path)
print(len(entries))
corpus = ""
for entry in entries:
  with open(sotu_path+entry,'r') as f:
    content = f.read()
    corpus += content + '\n'
file_name = '/content/drive/My Drive/0-software_dev_notes/dsta-ai-course/sotu.txt'
with open(file_name,'w') as f:
  f.write(corpus)
print('corpus length:', len(corpus))

228
corpus length: 10602749


## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)

In [16]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='345M',
              steps=1000,
              restore_from='fresh',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Loading checkpoint models/345M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:13<00:00, 13.32s/it]


dataset has 2269622 tokens
Training...
[10 | 17.51] loss=2.95 avg=2.95
[20 | 26.20] loss=3.30 avg=3.13
[30 | 34.87] loss=2.79 avg=3.02
[40 | 43.54] loss=2.99 avg=3.01
[50 | 52.21] loss=2.94 avg=3.00
[60 | 60.89] loss=2.77 avg=2.96
[70 | 69.59] loss=2.77 avg=2.93
[80 | 78.27] loss=2.61 avg=2.89
[90 | 86.97] loss=3.16 avg=2.92
[100 | 95.66] loss=2.57 avg=2.88
[110 | 104.34] loss=2.85 avg=2.88
[120 | 113.00] loss=2.65 avg=2.86
[130 | 121.68] loss=3.01 avg=2.87
[140 | 130.35] loss=3.03 avg=2.88
[150 | 139.02] loss=2.97 avg=2.89
[160 | 147.70] loss=2.94 avg=2.89
[170 | 156.38] loss=3.35 avg=2.92
[180 | 165.04] loss=2.71 avg=2.91
[190 | 173.71] loss=2.65 avg=2.89
[200 | 182.38] loss=2.98 avg=2.90
======== SAMPLE 1 ========
 months before the completion of the work, and the payment of the full sum of money appropriated by the Congress has been made which has been received;

but our own circumstances permit me to state and declare that the necessary funds are sufficient to discharge all the bo

In [17]:
ls checkpoint/run1

checkpoint    events.out.tfevents.1591028663.aa8f63524591  model-1000.index
counter       hparams.json                                 model-1000.meta
encoder.json  model-1000.data-00000-of-00001               vocab.bpe


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. (NB: if you are downloading the model to your personal computer, download the large model checkpoint file *seperately*, download the other files, and reconstruct the `/checkpoint/run1` folder hierarchy locally).

In [0]:
gpt2.copy_checkpoint_to_gdrive()

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `checkpoint` folder from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive()

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [1]:
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Colocations handled automatically by placer.
Loading checkpoint checkpoint/run1/model-1000
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [2]:
gpt2.generate(sess)

When asked to come to the defense of the United States during our
First World War, America did not hesitate to do so. Our armed forces in
Europe and Asia were in a state of great readiness, and so was
our navy. In the past year, however, the American ships in
the Mediterranean have been forced to make frequent port attacks on
the German Navy, and its own force is now being
deprived of the ships in the Mediterranean.

The present government has been in power for a few months, and
has already suffered many reverses. A year ago there was a
thrust of the utmost probity and confidence in the American
people. The public confidence in our nation was so great that
no one expected that we would fail. The peace and war were
inextricably bound together by the great principles of
liberty and justice.

The war in Europe had been fought by Germany, but the
people of Germany had not been able to form a strong
unified opinion of the war and of the war's result. It was
clear that Germany would have to 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [3]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Gentlemen ",
              nsamples=5,
              batch_size=5
              )

Gentlemen  as follows, and that they may be
observed by any one who may be in our country.

On the 10th of June last the Reverend Father of the
Church of the Nazarene and the Reverend Preacher of the
Nazarene Church were delivered over to the custody of the
Governor of the United States, and on the 30th of
January last the Reverend Father was again delivered to the
Governor, and on the 1st of March last the Reverend
Father was delivered to the Governor, and on the 30th of
March last, and on the 1st of September last, the Governor
delivered the Reverend Father back to the States of the
Americas, and on the 30th of September last, and on the
30th of September last, the Reverend Father was delivered to the
Governor, and on the 1st of October, and on the 31st of
September, the Governor delivered him over to the States of
the Americas, and on the 1st of October, and on the
31st of September, the Reverend Father was delivered over to the
States of the Americas, and on the 1st of October, and

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp and then download it.

You can rerun the cell as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

files.download(gen_file)